# Scrape the table from the Wikipedia page

In [187]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib

html = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_areas_of_London").read()
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
columns = [th.text.replace('\n', '') 
for th in table.find('tr').find_all('th')]
trs = table.find_all('tr')[1:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
df = pd.DataFrame(data=rows, columns=columns)
df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [1]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[2]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[2],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[2],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


# Wrangle and clean the data

#### Rename and drop some columns

In [188]:
df.columns = ['Location', 'Borough', 'Posttown', 'Postcode', 'Dialcode', 'OSgridref']
df = df.drop(columns = 'Posttown')
df = df.drop(columns = 'Dialcode')
df = df.drop(columns = 'OSgridref')
df.head()

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich [1]",SE2
1,Acton,"Ealing, Hammersmith and Fulham[2]","W3, W4"
2,Addington,Croydon[2],CR0
3,Addiscombe,Croydon[2],CR0
4,Albany Park,Bexley,"DA5, DA14"


#### Remove references in the 'Borough' column

In [189]:
df['Borough'] = df['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df.head()

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Acton,"Ealing, Hammersmith and Fulham","W3, W4"
2,Addington,Croydon,CR0
3,Addiscombe,Croydon,CR0
4,Albany Park,Bexley,"DA5, DA14"


#### Spread postcodes to multi rows if there are more than 1 postcodes (keep other column values the same)

In [190]:
df = df.drop('Postcode', axis=1).join(df['Postcode'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postcode')).reset_index(drop=True)
# df = df.drop(columns = 'index')
df.head()

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Acton,"Ealing, Hammersmith and Fulham",W3
2,Acton,"Ealing, Hammersmith and Fulham",W4
3,Addington,Croydon,CR0
4,Addiscombe,Croydon,CR0


#### Combine the rows whose 'Location' shares the same postcode

In [191]:
df = df.groupby(['Postcode','Borough'])['Location'].apply(', '.join).reset_index()
df.head()

,Postcode,Borough,Location
0,BR5,"Bexley, Bromley","Ruxley, Upper Ruxley"
1,BR6,Bromley,Orpington
2,CR0,Sutton,Beddington
3,DA14,Bexley,"Albany Park, Longlands"
4,DA15,Bexley,"Longlands, Sidcup"


#### Only consider North London Areas (Postcode starts with N)

In [192]:
df.Postcode = df.Postcode.str.strip()
df = df[df['Postcode'].str.startswith(('N'))].reset_index(drop=True)
df.head()

,Postcode,Borough,Location
0,N1,Hackney,Hackney
1,N1,Islington,Angel
2,N10,Barnet,Colney Hatch
3,N12,Barnet,Finchley
4,N14,Enfield,Arnos Grove


In [193]:
df.shape

(65, 3)

# Get the latitude and longitude of each location

#### Define a function to obtain latitude and longitude

In [194]:
!pip -q install geocoder
import geocoder

In [195]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

#### Obtain the data of latitude and longitude and append them to the dataframe

In [196]:
postcodes = df['Postcode']    
coordinates = [get_latlng(postcodes) for postcodes in postcodes.tolist()]

In [197]:
df_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df['Latitude'] = df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']
df.head()

,Postcode,Borough,Location,Latitude,Longitude
0,N1,Hackney,Hackney,51.53791,-0.09984
1,N1,Islington,Angel,51.53791,-0.09984
2,N10,Barnet,Colney Hatch,51.59107,-0.14852
3,N12,Barnet,Finchley,51.61592,-0.17674
4,N14,Enfield,Arnos Grove,51.63429,-0.13366


# Explore the locations in North London

#### Take an overview

In [198]:
print('The dataframe has {} boroughs and {} locations.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 14 boroughs and 65 locations.


#### Create a map of London with North London Locations superimposed on top

In [199]:
address = 'London'
geolocator = Nominatim(user_agent="lo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [200]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [201]:
map_London = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, location in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Location']):
    label = '{}, {}'.format(location, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_London)  
    
map_London

#### Define Foursquare Credentials and Version (Hidden Cell)

In [202]:
# The code was removed by Watson Studio for sharing.

#### Explore the Location Angel in the dataframe
#### And get the top 100 venues that are in Angel within a radius of 1000 meters

In [203]:
Angel_latitude = df.loc[1, 'Latitude']
Angel_longitude = df.loc[1, 'Longitude']

In [204]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Angel_latitude, 
    Angel_longitude, 
    radius, 
    LIMIT)
url

import requests

results = requests.get(url).json()

In [205]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [206]:
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,69 Colebrooke Row,Cocktail Bar,51.536480,-0.101207
1,Steve Hatt,Fish Market,51.538588,-0.099041
2,The Life Centre Islington,Yoga Studio,51.538056,-0.099171
3,Cass Art,Arts & Crafts Store,51.536379,-0.101013
4,The Bill Murray,Pub,51.536226,-0.098750


In [207]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [208]:
nearby_venues_unique = nearby_venues['categories'].value_counts().to_frame(name='count')
nearby_venues_unique.head(10)

,count
Pub,11
Mediterranean Restaurant,4
Gastropub,4
Burger Joint,4
French Restaurant,4
Bakery,3
Cocktail Bar,3
Boutique,3
Japanese Restaurant,2
Furniture / Home Store,2


#### Explore Locations in North London

In [209]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'Location Latitude', 
                  'Location Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [210]:
North_London_venues = getNearbyVenues(names=df['Location'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Hackney
Angel
Colney Hatch
Finchley
Arnos Grove
Harringay
Hackney, Shacklewell, Stamford Hill
South Tottenham, Tottenham, Tottenham Hale
Edmonton
Tufnell Park
Bounds Green
Finchley
Harringay
Neasden
Gospel Oak, Primrose Hill
Brent Cross
Kensal Green
Arkley
Primrose Hill
Hoxton, Shoreditch
Barnsbury, Canonbury, De Beauvoir Town, Islington, Pentonville
Fortis Green
Muswell Hill
Brunswick Park, Colney Hatch, Friern Barnet, New Southgate
Arnos Grove
Bounds Green
North Finchley, Woodside Park
Palmers Green
Osidge
Southgate
Stamford Hill
Seven Sisters, South Tottenham, Tottenham, Tottenham Green, Tottenham Hale, West Green
Stoke Newington, West Hackney
Archway, Upper Holloway
East Finchley, Finchley, Hampstead Garden Suburb
Oakleigh Park, Totteridge, Whetstone
Grange Park, Winchmore Hill
Bowes Park, Wood Green
Church End
Manor House
Harringay, Stroud Green
Finsbury Park
Highbury
Highgate
Holloway, Nag's Head, Tufnell Park
Crouch End, Hornsey, Turnpike Lane
Edmonton
Camden Town, Chalk Farm, P

In [211]:
print(North_London_venues.shape)
North_London_venues.head()

(3715, 7)


,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hackney,51.53791,-0.09984,69 Colebrooke Row,51.536480,-0.101207,Cocktail Bar
1,Hackney,51.53791,-0.09984,Steve Hatt,51.538588,-0.099041,Fish Market
2,Hackney,51.53791,-0.09984,The Life Centre Islington,51.538056,-0.099171,Yoga Studio
3,Hackney,51.53791,-0.09984,Cass Art,51.536379,-0.101013,Arts & Crafts Store
4,Hackney,51.53791,-0.09984,The Bill Murray,51.536226,-0.098750,Pub


In [212]:
North_London_venues.groupby('Location').count()

,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Location,,,,,,
Angel,100,100,100,100,100,100
"Archway, Upper Holloway",44,44,44,44,44,44
Arkley,19,19,19,19,19,19
Arnos Grove,52,52,52,52,52,52
"Barnet Gate, The Hale, Mill Hill",19,19,19,19,19,19
"Barnsbury, Canonbury, De Beauvoir Town, Islington, Pentonville",100,100,100,100,100,100
"Belsize Park, Frognal, Hampstead, Swiss Cottage",100,100,100,100,100,100
Bounds Green,97,97,97,97,97,97
"Bowes Park, Wood Green",77,77,77,77,77,77


In [213]:
print('There are {} uniques categories.'.format(len(North_London_venues['Venue Category'].unique())))

There are 231 uniques categories.


In [214]:
North_London_venues_unique = North_London_venues['Venue Category'].value_counts().to_frame(name='count')
North_London_venues_unique.head(10)

,count
Pub,314
Café,249
Coffee Shop,223
Grocery Store,163
Pizza Place,122
Gym / Fitness Center,104
Bakery,98
Italian Restaurant,95
Park,89
Indian Restaurant,68


#### Analyze Each Location

In [215]:
North_London_onehot = pd.get_dummies(North_London_venues[['Venue Category']], prefix="", prefix_sep="")

North_London_onehot['Location'] = North_London_venues['Location'] 

fixed_columns = [North_London_onehot.columns[-1]] + list(North_London_onehot.columns[:-1])
North_London_onehot = North_London_onehot[fixed_columns]

North_London_onehot.head()

,Location,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,...,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Hackney,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hackney,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hackney,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Hackney,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hackney,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [216]:
North_London_onehot.shape

(3715, 232)

#### Group rows by location and by taking the mean of the frequency of occurrence of each category

In [217]:
North_London_grouped = North_London_onehot.groupby('Location').mean().reset_index()
North_London_grouped

,Location,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,...,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Angel,0.01,0.000000,0.01,0.000000,0.00,0.020000,0.000000,0.000000,0.01,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.01,0.010000,0.00,0.00
1,"Archway, Upper Holloway",0.00,0.000000,0.00,0.000000,0.00,0.000000,0.022727,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.022727,0.00,0.00,0.000000,0.00,0.00
2,Arkley,0.00,0.000000,0.00,0.052632,0.00,0.000000,0.000000,0.052632,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
3,Arnos Grove,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
4,"Barnet Gate, The Hale, Mill Hill",0.00,0.000000,0.00,0.052632,0.00,0.000000,0.000000,0.052632,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
5,"Barnsbury, Canonbury, De Beauvoir Town, Isling...",0.01,0.000000,0.01,0.000000,0.00,0.020000,0.000000,0.000000,0.01,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.01,0.010000,0.00,0.00
6,"Belsize Park, Frognal, Hampstead, Swiss Cottage",0.00,0.020000,0.00,0.010000,0.01,0.010000,0.010000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00
7,Bounds Green,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.010309,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
8,"Bowes Park, Wood Green",0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.012987,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00
9,Brent Cross,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00


In [218]:
North_London_grouped.shape

(60, 232)

#### Print each location along with the top 5 most common venues

In [219]:
num_top_venues = 5

for hood in North_London_grouped['Location']:
    print("----"+hood+"----")
    temp = North_London_grouped[North_London_grouped['Location'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Angel----
                      venue  freq
0                       Pub  0.11
1  Mediterranean Restaurant  0.04
2              Burger Joint  0.04
3                 Gastropub  0.04
4         French Restaurant  0.04


----Archway, Upper Holloway----
                venue  freq
0                 Pub  0.20
1         Coffee Shop  0.09
2         Pizza Place  0.07
3  Italian Restaurant  0.07
4               Hotel  0.05


----Arkley----
                  venue  freq
0         Grocery Store  0.11
1  Fast Food Restaurant  0.05
2     Indian Restaurant  0.05
3             Bookstore  0.05
4  Gym / Fitness Center  0.05


----Arnos Grove----
                  venue  freq
0         Grocery Store  0.08
1           Coffee Shop  0.06
2  Fast Food Restaurant  0.06
3  Gym / Fitness Center  0.06
4                   Pub  0.04


----Barnet Gate, The Hale, Mill Hill----
                  venue  freq
0         Grocery Store  0.11
1  Fast Food Restaurant  0.05
2     Indian Restaurant  0.05
3             Book

#### Put this into a pandas dataframe

In [220]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [221]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Location']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

locations_venues_sorted = pd.DataFrame(columns=columns)
locations_venues_sorted['Location'] = North_London_grouped['Location']

for ind in np.arange(North_London_grouped.shape[0]):
    locations_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_London_grouped.iloc[ind, :], num_top_venues)

locations_venues_sorted.head()

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angel,Pub,French Restaurant,Mediterranean Restaurant,Burger Joint,Gastropub,Boutique,Cocktail Bar,Bakery,Arts & Crafts Store,Coffee Shop
1,"Archway, Upper Holloway",Pub,Coffee Shop,Pizza Place,Italian Restaurant,Hotel,Gastropub,Café,Park,Eastern European Restaurant,Music Venue
2,Arkley,Grocery Store,Discount Store,Train Station,Café,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Bookstore,Salon / Barbershop,Gym / Fitness Center
3,Arnos Grove,Grocery Store,Fast Food Restaurant,Gym / Fitness Center,Coffee Shop,Bus Stop,Pizza Place,Park,Electronics Store,Café,Pub
4,"Barnet Gate, The Hale, Mill Hill",Grocery Store,Discount Store,Train Station,Café,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Bookstore,Salon / Barbershop,Gym / Fitness Center


# Cluster Locations

In [222]:
from sklearn.cluster import KMeans

kclusters = 5

North_London_grouped_clustering = North_London_grouped.drop('Location', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_London_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 2, 1, 1, 1, 2, 3, 1, 1, 4], dtype=int32)

In [223]:
locations_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

North_London_merged = df

North_London_merged = North_London_merged.join(locations_venues_sorted.set_index('Location'), on='Location')

North_London_merged.head()

,Postcode,Borough,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,N1,Hackney,Hackney,51.53791,-0.09984,2,Pub,French Restaurant,Mediterranean Restaurant,Burger Joint,Gastropub,Boutique,Cocktail Bar,Bakery,Arts & Crafts Store,Coffee Shop
1,N1,Islington,Angel,51.53791,-0.09984,2,Pub,French Restaurant,Mediterranean Restaurant,Burger Joint,Gastropub,Boutique,Cocktail Bar,Bakery,Arts & Crafts Store,Coffee Shop
2,N10,Barnet,Colney Hatch,51.59107,-0.14852,3,Café,Coffee Shop,Pizza Place,Pub,Bakery,English Restaurant,Deli / Bodega,Grocery Store,Gym / Fitness Center,Forest
3,N12,Barnet,Finchley,51.61592,-0.17674,4,Coffee Shop,Supermarket,Café,Indian Restaurant,Turkish Restaurant,Japanese Restaurant,Pub,Gym / Fitness Center,Fast Food Restaurant,Park
4,N14,Enfield,Arnos Grove,51.63429,-0.13366,1,Grocery Store,Fast Food Restaurant,Gym / Fitness Center,Coffee Shop,Bus Stop,Pizza Place,Park,Electronics Store,Café,Pub


In [224]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(North_London_merged['Latitude'], North_London_merged['Longitude'], North_London_merged['Location'], North_London_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters
#### Cluster 1

In [225]:
North_London_merged.loc[North_London_merged['Cluster Labels'] == 0, North_London_merged.columns[[1] + list(range(5, North_London_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Barnet,0,Hookah Bar,Asian Restaurant,Pub,Costume Shop,Supermarket,Gas Station,Fast Food Restaurant,Outdoor Supply Store,Coffee Shop,Café
64,Brent,0,Hookah Bar,Asian Restaurant,Pub,Costume Shop,Supermarket,Gas Station,Fast Food Restaurant,Outdoor Supply Store,Coffee Shop,Café


#### Cluster 2

In [226]:
North_London_merged.loc[North_London_merged['Cluster Labels'] == 1, North_London_merged.columns[[1] + list(range(5, North_London_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Enfield,1,Grocery Store,Fast Food Restaurant,Gym / Fitness Center,Coffee Shop,Bus Stop,Pizza Place,Park,Electronics Store,Café,Pub
7,Haringey,1,Pub,Gym / Fitness Center,Bar,Supermarket,Café,Grocery Store,Convenience Store,Pizza Place,Train Station,Indie Theater
10,Haringey,1,Pub,Grocery Store,Café,Coffee Shop,Fast Food Restaurant,Clothing Store,Park,Chinese Restaurant,Bakery,Greek Restaurant
13,Brent,1,Platform,Grocery Store,Plaza,Pub,Fast Food Restaurant,Coffee Shop,Hotel,Sandwich Place,Bus Stop,Café
17,Barnet,1,Grocery Store,Discount Store,Train Station,Café,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Bookstore,Salon / Barbershop,Gym / Fitness Center
23,Barnet,1,Grocery Store,Park,Hardware Store,Sporting Goods Shop,Nature Preserve,Chinese Restaurant,Furniture / Home Store,Coffee Shop,Fast Food Restaurant,Construction & Landscaping
24,Enfield,1,Grocery Store,Fast Food Restaurant,Gym / Fitness Center,Coffee Shop,Bus Stop,Pizza Place,Park,Electronics Store,Café,Pub
25,Haringey,1,Pub,Grocery Store,Café,Coffee Shop,Fast Food Restaurant,Clothing Store,Park,Chinese Restaurant,Bakery,Greek Restaurant
27,Enfield,1,Grocery Store,Greek Restaurant,Bookstore,Pub,Park,Indian Restaurant,Fast Food Restaurant,Coffee Shop,Tennis Court,Tea Room
28,Barnet,1,Gym / Fitness Center,Fast Food Restaurant,Pizza Place,Grocery Store,Pub,Coffee Shop,Smoke Shop,Steakhouse,Bus Station,Bus Stop


#### Cluster 3

In [227]:
North_London_merged.loc[North_London_merged['Cluster Labels'] == 2, North_London_merged.columns[[1] + list(range(5, North_London_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hackney,2,Pub,French Restaurant,Mediterranean Restaurant,Burger Joint,Gastropub,Boutique,Cocktail Bar,Bakery,Arts & Crafts Store,Coffee Shop
1,Islington,2,Pub,French Restaurant,Mediterranean Restaurant,Burger Joint,Gastropub,Boutique,Cocktail Bar,Bakery,Arts & Crafts Store,Coffee Shop
9,Islington,2,Pub,Coffee Shop,Pizza Place,Italian Restaurant,Hotel,Gastropub,Café,Park,Eastern European Restaurant,Music Venue
19,Hackney,2,Pub,French Restaurant,Mediterranean Restaurant,Burger Joint,Gastropub,Boutique,Cocktail Bar,Bakery,Arts & Crafts Store,Coffee Shop
20,Islington,2,Pub,French Restaurant,Mediterranean Restaurant,Burger Joint,Gastropub,Boutique,Cocktail Bar,Bakery,Arts & Crafts Store,Coffee Shop
33,Islington,2,Pub,Coffee Shop,Pizza Place,Italian Restaurant,Hotel,Gastropub,Café,Park,Eastern European Restaurant,Music Venue
43,Camden,2,Pub,Café,Pizza Place,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Bus Stop,Park,Trail
47,Camden,2,Zoo Exhibit,Pub,Coffee Shop,Music Venue,Market,Greek Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Garden,Bar


#### Cluster 4

In [228]:
North_London_merged.loc[North_London_merged['Cluster Labels'] == 3, North_London_merged.columns[[1] + list(range(5, North_London_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Barnet,3,Café,Coffee Shop,Pizza Place,Pub,Bakery,English Restaurant,Deli / Bodega,Grocery Store,Gym / Fitness Center,Forest
5,Haringey,3,Café,Pub,Grocery Store,Coffee Shop,Pizza Place,Bus Stop,Park,Bakery,Gym / Fitness Center,Thai Restaurant
6,Hackney,3,Pub,Café,Cocktail Bar,Turkish Restaurant,Coffee Shop,Japanese Restaurant,Pizza Place,Thai Restaurant,Italian Restaurant,Grocery Store
12,Haringey,3,Café,Pub,Grocery Store,Coffee Shop,Pizza Place,Bus Stop,Park,Bakery,Gym / Fitness Center,Thai Restaurant
14,Camden,3,Café,Pub,Bakery,Italian Restaurant,Pizza Place,Coffee Shop,Bookstore,Museum,Gym / Fitness Center,Movie Theater
16,Brent,3,Coffee Shop,Pub,Grocery Store,Indian Restaurant,Pizza Place,Thai Restaurant,Bar,Bakery,Café,Middle Eastern Restaurant
18,Camden,3,Café,Pub,Coffee Shop,Deli / Bodega,Grocery Store,Thai Restaurant,Cricket Ground,French Restaurant,Garden,Bakery
21,Haringey,3,Café,Coffee Shop,Pizza Place,Pub,Bakery,English Restaurant,Deli / Bodega,Grocery Store,Gym / Fitness Center,Forest
22,Haringey and Barnet,3,Café,Coffee Shop,Pizza Place,Pub,Bakery,English Restaurant,Deli / Bodega,Grocery Store,Gym / Fitness Center,Forest
32,Hackney,3,Pub,Café,Cocktail Bar,Turkish Restaurant,Coffee Shop,Japanese Restaurant,Pizza Place,Thai Restaurant,Italian Restaurant,Grocery Store


#### Cluster 5

In [229]:
North_London_merged.loc[North_London_merged['Cluster Labels'] == 4, North_London_merged.columns[[1] + list(range(5, North_London_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Barnet,4,Coffee Shop,Supermarket,Café,Indian Restaurant,Turkish Restaurant,Japanese Restaurant,Pub,Gym / Fitness Center,Fast Food Restaurant,Park
8,Enfield,4,Turkish Restaurant,Supermarket,Coffee Shop,Pub,Fast Food Restaurant,Hotel,Café,Sandwich Place,Convenience Store,Bakery
11,Barnet,4,Coffee Shop,Supermarket,Café,Indian Restaurant,Turkish Restaurant,Japanese Restaurant,Pub,Gym / Fitness Center,Fast Food Restaurant,Park
15,Barnet,4,Coffee Shop,Grocery Store,Pizza Place,Hotel,Gym / Fitness Center,Korean Restaurant,Pub,Park,Sandwich Place,Café
26,Barnet,4,Coffee Shop,Café,Supermarket,Grocery Store,Bookstore,Gift Shop,Theater,Thai Restaurant,Tea Room,Sushi Restaurant
35,Barnet,4,Coffee Shop,Pub,Italian Restaurant,Hotel,Park,Pizza Place,Brazilian Restaurant,Supermarket,Mediterranean Restaurant,Bistro
46,Enfield,4,Turkish Restaurant,Supermarket,Coffee Shop,Pub,Fast Food Restaurant,Hotel,Café,Sandwich Place,Convenience Store,Bakery
51,Barnet,4,Coffee Shop,Grocery Store,Café,Bakery,Turkish Restaurant,Korean Restaurant,Supermarket,Italian Restaurant,Chinese Restaurant,Sushi Restaurant
56,Barnet,4,Coffee Shop,Grocery Store,Pizza Place,Hotel,Gym / Fitness Center,Korean Restaurant,Pub,Park,Sandwich Place,Café
